In [3]:
import os.path as osp

import torch
import torch.nn.functional as F

from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GraphConv, TopKPooling
from torch_geometric.nn import global_max_pool as gmp
from torch_geometric.nn import global_mean_pool as gap

path = 'data/TUDataset'
dataset = TUDataset(path, name='PROTEINS')
dataset = dataset.shuffle()
n = len(dataset) // 10
test_dataset = dataset[:n]
train_dataset = dataset[n:]
test_loader = DataLoader(test_dataset, batch_size=60)
train_loader = DataLoader(train_dataset, batch_size=60)


class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = GraphConv(dataset.num_features, 128)
        self.pool1 = TopKPooling(128, ratio=0.8)
        self.conv2 = GraphConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.8)
        self.conv3 = GraphConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.8)

        self.lin1 = torch.nn.Linear(256, 128)
        self.lin2 = torch.nn.Linear(128, 64)
        self.lin3 = torch.nn.Linear(64, dataset.num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        x = F.relu(self.conv1(x, edge_index))
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))
        x, edge_index, _, batch, _, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.lin2(x))
        x = F.log_softmax(self.lin3(x), dim=-1)

        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)


def train(epoch):
    model.train()

    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, data.y)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)


def test(loader):
    model.eval()

    correct = 0
    for data in loader:
        data = data.to(device)
        pred = model(data).max(dim=1)[1]
        correct += pred.eq(data.y).sum().item()
    return correct / len(loader.dataset)


for epoch in range(1, 201):
    loss = train(epoch)
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.5f}, Train Acc: {train_acc:.5f}, '
          f'Test Acc: {test_acc:.5f}')

Epoch: 001, Loss: 0.63994, Train Acc: 0.67166, Test Acc: 0.77477
Epoch: 002, Loss: 0.60602, Train Acc: 0.72555, Test Acc: 0.75676
Epoch: 003, Loss: 0.59269, Train Acc: 0.73553, Test Acc: 0.78378
Epoch: 004, Loss: 0.57202, Train Acc: 0.73353, Test Acc: 0.75676
Epoch: 005, Loss: 0.56646, Train Acc: 0.73553, Test Acc: 0.72973
Epoch: 006, Loss: 0.56377, Train Acc: 0.74451, Test Acc: 0.76577
Epoch: 007, Loss: 0.53971, Train Acc: 0.74950, Test Acc: 0.77477
Epoch: 008, Loss: 0.54838, Train Acc: 0.74750, Test Acc: 0.76577
Epoch: 009, Loss: 0.54948, Train Acc: 0.75349, Test Acc: 0.75676
Epoch: 010, Loss: 0.53813, Train Acc: 0.75349, Test Acc: 0.76577
Epoch: 011, Loss: 0.53843, Train Acc: 0.75150, Test Acc: 0.76577
Epoch: 012, Loss: 0.53526, Train Acc: 0.75649, Test Acc: 0.74775
Epoch: 013, Loss: 0.53497, Train Acc: 0.76347, Test Acc: 0.76577
Epoch: 014, Loss: 0.52893, Train Acc: 0.76447, Test Acc: 0.76577
Epoch: 015, Loss: 0.52412, Train Acc: 0.75848, Test Acc: 0.74775
Epoch: 016, Loss: 0.52143